# Setup

In [1]:
# !pip install webdriver-manager


In [32]:
import csv
import os
import pathlib
import re
import time
import urllib.request
from urllib.parse import urlparse
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common import keys
from selenium.webdriver.chrome.service import Service as BraveService
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.core.utils import ChromeType
import selenium

In [33]:
class Paths:
    WORK_DIR = pathlib.Path().absolute()
    TEST_DATA_DIR = WORK_DIR.parent / "nfdi4bio"
    TEST_DATASET = TEST_DATA_DIR / "wuchsbezirke_2011.zip"
    TEST_METADATA_CSV = TEST_DATA_DIR / "wuchsbezirka_metadata.csv"
    OUR_DOWNLOADED_METADATA = WORK_DIR / "Wuchsbezirke 2011_ISO_Metadata"
    OUR_DOWNLOADED_METADATA_THROUGH_API = WORK_DIR / "Wuchsbezirke 2011_ISO_Metadata_with_api"
    ORIGINAL_DOWNLOADED_METADATA = WORK_DIR / "original_metadata"
    
ROOT_URL = "http://localhost/"
    
class Login:
    NAME = "admin"
    PASSWORD = "admin"

In [23]:
chrome_options = Options()
chrome_options.add_argument('--no-sandbox')
# chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--profile-directory=Default')
chrome_options.add_argument('--user-data-dir=~/.config/google-chrome')
chrome_options.add_argument('start-maximized')
prefs = {"download.default_directory" : Paths.WORK_DIR.as_posix()}
chrome_options.add_experimental_option("prefs",prefs)

In [24]:
driver = webdriver.Chrome(service=BraveService(ChromeDriverManager(chrome_type=ChromeType.BRAVE).install()),chrome_options=chrome_options)

/tmp/ipykernel_8397/1788188964.py:1: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(service=BraveService(ChromeDriverManager(chrome_type=ChromeType.BRAVE).install()),chrome_options=chrome_options)


In [25]:
def hide_footer(func):
    def wrap(*args, **kwargs):
        out = func(*args, **kwargs)
        footer = driver.find_element(By.XPATH,"//footer")
        driver.execute_script("arguments[0].style.visibility='hidden'",footer)
        return out
    return wrap

## Set Page To English

In [26]:
def set_page_to_english():
    def already_english():
        return driver.find_element(By.ID, "gn-language-selector").text == "English"

    def open_dropdown_with_languages():
        driver.find_element(By.ID, "gn-language-selector").click()

    def click_english_language_element():
        driver.find_element(By.XPATH, '//*[@id="gn-language-selector-dropdown"]/ul/li/a[text()="English"]').click()

    driver.get(ROOT_URL)
    if already_english():
        return
    open_dropdown_with_languages()
    click_english_language_element()

set_page_to_english()

## Login

In [27]:
def login():
    LOGIN_URL = ROOT_URL + "account/login/"
    @hide_footer
    def load_login_page():
        TIMEOUT = 10
        driver.get(LOGIN_URL)
        form_has_loaded = EC.presence_of_element_located((By.CSS_SELECTOR, "form"))
        try:
            WebDriverWait(driver, TIMEOUT).until(form_has_loaded)
        except TimeoutException as e:
            raise e("login form didn't load in time.")

    def input_name():
        input = driver.find_element(By.CSS_SELECTOR, "#id_login")
        input.send_keys(Login.NAME)

    def input_password():
        input = driver.find_element(By.CSS_SELECTOR, "#id_password")
        input.send_keys(Login.PASSWORD)

    def click_login():
        driver.find_element(By.XPATH, '//form/button[text()="Sign In"]').click()

    load_login_page()
    input_name()
    input_password()
    click_login()
    
login()

# Step 1

In [9]:
def upload_dataset_and_get_its_link():
    UPLOAD_DATASET_URL = ROOT_URL + "catalogue/#/upload/dataset"
    @hide_footer
    def navigate_to_the_dataset_upload_UI():
        TIMEOUT = 10
        driver.get(UPLOAD_DATASET_URL)
        UI_has_loaded = EC.presence_of_element_located((By.ID, "ms-app"))
        try:
            WebDriverWait(driver, TIMEOUT).until(UI_has_loaded)
        except TimeoutException as e:
            raise e("upload app didn't load in time.")

    def select_the_dataset_to_be_uploaded():
        input = driver.find_element(By.XPATH, '//*[@id="ms-app"]/div/div/div[1]/div[1]/div/div[1]/input')
        input.send_keys(Paths.TEST_DATASET.as_posix())

    def click_on_upload():
        button = driver.find_element(By.XPATH, '//*[@id="ms-app"]/div/div/div[1]/div[1]/div/div[2]/button')
        button.click()

    def wait_until_the_upload_process_is_finalized():
        TIMEOUT = 240
        try:
            show_button_is_present = EC.presence_of_element_located((By.XPATH, "//a[span/text()='View']"))
            WebDriverWait(driver, TIMEOUT).until(show_button_is_present)
        except TimeoutException as e:
            raise e("Upload of dataset didn't succeed in time.")

    def get_link_to_uploaded_dataset():
        return driver.find_element(By.XPATH, "//a[span/text()='View']").get_attribute("href")

    navigate_to_the_dataset_upload_UI()
    select_the_dataset_to_be_uploaded()
    click_on_upload()
    wait_until_the_upload_process_is_finalized()
    return get_link_to_uploaded_dataset()

dataset_layer_url = upload_dataset_and_get_its_link()


# Step 2A

In [18]:
# link needs to be normalized for remote debugging:
# dataset_layer_url= urlparse(dataset_layer_url)._replace(netloc=urlparse(ROOT_URL).netloc).geturl()  
# dataset_layer_url="http://localhost/catalogue/#/dataset/8"

In [16]:
def open_editor_edit_metadata_and_update():
    @hide_footer
    def navigate_to_the_new_dataset_layer():
        driver.get(dataset_layer_url)

    def click_on_edit_metadata():
        def close_overlaying_UI():
            TIMEOUT = 30
            is_loaded = EC.presence_of_element_located((By.XPATH, '//a[@class="square-button btn btn-default" and @role="button"]'))
            try:
                WebDriverWait(driver, TIMEOUT).until(is_loaded)
            except TimeoutException as e:
                raise e("page didn't load in time.")
            driver.find_element(By.XPATH, '//a[@class="square-button btn btn-default" and @role="button"]').click()

        close_overlaying_UI()
        driver.find_element(By.XPATH, "//button[span/text()='Edit']").click()
        driver.find_element(By.XPATH, '//ul/li/a[span/text()="Edit Metadata"]').click()

    @hide_footer
    def click_on_advanced_metadata():
        driver.find_element(By.XPATH, '//a[text()="Advanced Metadata"]').click()

    def load_metadatas_from_csv():
        with open(Paths.TEST_METADATA_CSV, mode="r", encoding="utf8") as csvfile:
            reader = csv.reader(csvfile, delimiter=";")
            metadatas = [(name, value.strip()) for name, value in reader][1:] # [1:] excludes header
        return metadatas

    def make_input_string(values):
        input_string = values[0] if len(values) == 1 else ", ".join(values) + ","
        return input_string

    def insert_the_metadatas_values_to_form(metadatas):
        for name, value in metadatas:
            # insert_one_metadata_value_to_form(field_name, input_string)
            match name:
                case "Title":
                    element = driver.find_element(By.XPATH,'//*[@id="id_resource-title"]')
                    element.clear()
                    element.send_keys(value)
                    
                case "Abstract":
                    iframe = element.find_element(By.XPATH, '//*[@id="id_resource-abstract_ifr"]')
                    driver.switch_to.frame(iframe)
                    element = driver.find_element(By.ID,"tinymce")
                    element.click()
                    element.clear()
                    element.send_keys(value)
                    driver.switch_to.default_content()
                    
                case "Type":
                    pass #TODO
                
                case "Category":
                    element = driver.find_element(By.XPATH,'//*[@id="category_form"]/div[5]/label')
                    element.click()
                    
                case "Organisation":
                    pass #TODO
                
                case "Creator":
                    pass #TODO
                
                case "Publisher":
                    pass #TODO
                
                case "Contributor":
                    pass #TODO
                
                case "Language":
                    element = Select(driver.find_element(By.ID,'id_resource-language'))
                    element.select_by_value(value)
                    
                case "License":
                    element =  Select(driver.find_element(By.ID,'id_resource-license'))
                    #TODO the license is not in dropdown
                    # element.select_by_value(value)
                    
                case "Keyword":
                    element = driver.find_element(By.XPATH,'//*[@id="metadata_update"]/div[2]/div[11]/div/span[2]/span[1]/span/ul/li/input')
                    element.send_keys(value+",")
                    
    def get_title(metadatas):
        return [t[1] for t in metadatas if t[0]=="Title"][0]
    
    def finalize_by_clicking_on_update(title):
        driver.find_element(By.ID,"btn_upd_md_dwn").click()
        try:
            WebDriverWait(driver, 120).until(EC.title_contains(title))
        except TimeoutException as e:
            e("updating didnt finish in time")
    
    navigate_to_the_new_dataset_layer()
    click_on_edit_metadata()
    click_on_advanced_metadata()
    metadatas = load_metadatas_from_csv()
    insert_the_metadatas_values_to_form(metadatas)
    finalize_by_clicking_on_update(get_title(metadatas))
open_editor_edit_metadata_and_update()

# Step 3

In [28]:
# driver.get(dataset_layer_url)

In [29]:
def close_overlaying_UI():
    TIMEOUT = 30
    is_loaded = EC.presence_of_element_located((By.XPATH, '//a[@class="square-button btn btn-default" and @role="button"]'))
    try:
        WebDriverWait(driver, TIMEOUT).until(is_loaded)
    except TimeoutException as e:
        raise e("page didn't load in time.")
    driver.find_element(By.XPATH, '//a[@class="square-button btn btn-default" and @role="button"]').click()
close_overlaying_UI()

In [30]:
def download_the_metadata_via_the_UI_at_the_dataset_details_view():
    driver.find_element(By.XPATH, "//button[span/text()='Download']").click()
    driver.find_element(By.XPATH, '//ul/li/button[span/text()="ISO Metadata"]').click()
download_the_metadata_via_the_UI_at_the_dataset_details_view()

In [ ]:
driver.quit()

In [34]:
def open_the_file_and_extract_the_ID():
    with open(Paths.OUR_DOWNLOADED_METADATA) as f:
        text = f.read()
    match = re.search("<gmd:fileIdentifier>[^>]*>(?P<ID>[^<]*)",text)
    return match.group("ID")
id = open_the_file_and_extract_the_ID()
id

'9d3da60b-5367-4c0a-831f-0d13c02f891c'

In [37]:
def download_the_metadata_from_the_pycsw_interface(id):
    url = f"http://localhost/catalogue/csw?request=GetRecordById&service=CSW&version=2.0.2&outputschema=http%3A%2F%2Fwww.isotc211.org%2F2005%2Fgmd&elementsetname=full&id={id}"
    urllib.request.urlretrieve(url, Paths.OUR_DOWNLOADED_METADATA_THROUGH_API)
download_the_metadata_from_the_pycsw_interface(id)

In [38]:
def download_the_original_thuenen_metadata_from_the_pycsw_interface():
    url = "https://atlas.thuenen.de/catalogue/csw?request=GetRecordById&service=CSW&version=2.0.2&id=7cbc8af9-792c-464c-b1a0-7155f1088d9b&outputschema=http%3A%2F%2Fwww.isotc211.org%2F2005%2Fgmd&elementsetname=full"
    urllib.request.urlretrieve(url, Paths.ORIGINAL_DOWNLOADED_METADATA)

download_the_original_thuenen_metadata_from_the_pycsw_interface()

In [ ]:
def verify_that_the_contents_are_the_same():
    original = Paths.ORIGINAL_DOWNLOADED_METADATA
    our = Paths.OUR_DOWNLOADED_METADATA_THROUGH_API
    # TODO return True if contents are the same
    return False

In [36]:
def delete_downloaded_files():
    files_to_delete = [Paths.OUR_DOWNLOADED_METADATA, Paths.OUR_DOWNLOADED_METADATA_THROUGH_API, Paths.ORIGINAL_DOWNLOADED_METADATA]
    for file in files_to_delete:
        os.remove(file)
delete_downloaded_files()